In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats


db = wrds.Connection(wrds_username='dachxiu')

Loading library list...
Done


In [2]:
db.list_tables(library="comp")

['anncomp',
 'codirfin',
 'colev',
 'coperol',
 'deferredcomp',
 'directorcomp',
 'ex_black',
 'ex_header',
 'exnames',
 'ltawdtab',
 'outstandingawards',
 'pension',
 'person',
 'planbasedawards',
 'stgrttab',
 'filings',
 'co_offtitl',
 'g_chars',
 'g_co_aaudit',
 'g_co_adesind',
 'g_co_afnd1',
 'g_co_afnd2',
 'g_co_afnddc1',
 'g_co_afnddc2',
 'g_co_afntind1',
 'g_co_afntind2',
 'g_co_ainvval',
 'g_co_hgic',
 'g_co_iaudit',
 'g_co_idesind',
 'g_co_ifndq',
 'g_co_ifndsa',
 'g_co_ifndytd',
 'g_co_ifntq',
 'g_co_ifntsa',
 'g_co_ifntytd',
 'g_co_industry',
 'g_co_ipcd',
 'g_company',
 'g_co_offtitl',
 'g_currency',
 'g_ecind_desc',
 'g_ecind_mth',
 'g_exrt_dly',
 'g_exrt_mth',
 'g_funda',
 'g_funda_fncd',
 'g_fundq',
 'g_fundq_fncd',
 'g_idxcst_his',
 'g_idx_daily',
 'g_idx_index',
 'g_idx_mth',
 'g_names',
 'g_names_ix',
 'g_names_ix_cst',
 'g_namesq',
 'g_sec_adesind',
 'g_sec_adjfact',
 'g_sec_afnd',
 'g_sec_afnddc',
 'g_sec_afnt',
 'g_secd',
 'g_sec_divid',
 'g_sec_dprc',
 'g_sec_dtr

In [3]:
db.describe_table(library="comp", table="funda")

Approximately 804857 rows in comp.funda.


,name,nullable,type
0,gvkey,True,VARCHAR(6)
1,datadate,True,DATE
2,fyear,True,DOUBLE PRECISION
3,indfmt,True,VARCHAR(12)
4,consol,True,VARCHAR(2)
...,...,...,...
943,au,True,VARCHAR(8)
944,auop,True,VARCHAR(8)
945,auopic,True,VARCHAR(1)
946,ceoso,True,VARCHAR(1)


comp = conn.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1959'
                    """)

https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=7&file_id=95302

gvkey 	Char	6	Global Company Key
datadate 	Num	8	Data Date
at 	Num	8	Assets - Total
pstkl 	Num	8	Preferred Stock - Liquidating Value
seq 	Num	8	Stockholders Equity - Parent
pstk 	Num	8	Preferred/Preference Stock (Capital) - Total

In [4]:
comp = db.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/2015'
                    """)

In [5]:
comp['datadate']

0        2015-05-31
1        2016-05-31
2        2017-05-31
3        2018-05-31
4        2019-05-31
            ...    
45686    2016-12-31
45687    2017-12-31
45688    2018-12-31
45689    2017-09-30
45690    2018-09-30
Name: datadate, Length: 45691, dtype: object

In [6]:
comp['datadate']=pd.to_datetime(comp['datadate'])

In [7]:
comp['datadate']

0       2015-05-31
1       2016-05-31
2       2017-05-31
3       2018-05-31
4       2019-05-31
           ...    
45686   2016-12-31
45687   2017-12-31
45688   2018-12-31
45689   2017-09-30
45690   2018-09-30
Name: datadate, Length: 45691, dtype: datetime64[ns]

In [8]:
comp['year']=comp['datadate'].dt.year

In [9]:
comp['year']

0        2015
1        2016
2        2017
3        2018
4        2019
         ... 
45686    2016
45687    2017
45688    2018
45689    2017
45690    2018
Name: year, Length: 45691, dtype: int64

In [10]:
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])

In [11]:
comp['ps']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
45686    0.0
45687    0.0
45688    0.0
45689    0.0
45690    0.0
Name: ps, Length: 45691, dtype: float64

In [12]:
comp['txditc']=comp['txditc'].fillna(0)

In [13]:
comp['txditc']

0        104.6
1         34.3
2         37.2
3         15.7
4          0.0
         ...  
45686      0.9
45687     11.0
45688     58.3
45689      0.0
45690      0.0
Name: txditc, Length: 45691, dtype: float64

In [14]:
comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

In [15]:
comp['be']

0         949.700
1         900.100
2         951.400
3         952.000
4         905.900
           ...   
45686    1342.700
45687    1418.900
45688    1742.800
45689       8.494
45690       9.475
Name: be, Length: 45691, dtype: float64

In [18]:
comp=comp.sort_values(by=['gvkey','datadate'])

In [19]:
comp['count']=comp.groupby(['gvkey'])

ValueError: Length of values does not match length of index

In [20]:
comp['count']

KeyError: 'count'

In [21]:
comp['count']=comp.groupby(['gvkey']).cumcount()

In [22]:
comp['count']

0        0
1        1
2        2
3        3
4        4
        ..
45686    1
45687    2
45688    3
45689    0
45690    1
Name: count, Length: 45691, dtype: int64

In [23]:
comp=comp[['gvkey','datadate','year','be','count']]

In [24]:
comp

,gvkey,datadate,year,be,count
0,001004,2015-05-31,2015,949.700,0
1,001004,2016-05-31,2016,900.100,1
2,001004,2017-05-31,2017,951.400,2
3,001004,2018-05-31,2018,952.000,3
4,001004,2019-05-31,2019,905.900,4
...,...,...,...,...,...
45686,328795,2016-12-31,2016,1342.700,1
45687,328795,2017-12-31,2017,1418.900,2
45688,328795,2018-12-31,2018,1742.800,3
45689,330227,2017-09-30,2017,8.494,0


###第60行往下CRSP数据读取及处理

In [25]:
crsp_m = db.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/2015' and '01/01/2019'
                      and b.exchcd between 1 and 3
                      """) 

In [26]:
crsp_m

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc
0,10001.0,7953.0,2015-01-30,11.0,2.0,-0.115245,-0.115245,10488.0,9.750000
1,10001.0,7953.0,2015-02-27,11.0,2.0,0.035897,0.035897,10488.0,10.100000
2,10001.0,7953.0,2015-03-31,11.0,2.0,-0.000495,-0.013861,10452.0,9.960000
3,10001.0,7953.0,2015-04-30,11.0,2.0,0.011044,0.011044,10452.0,10.070000
4,10001.0,7953.0,2015-05-29,11.0,2.0,-0.006951,-0.006951,10488.0,10.000000
...,...,...,...,...,...,...,...,...,...
276086,17678.0,56330.0,2018-10-31,12.0,3.0,-0.212421,-0.212421,21010.0,30.715599
276087,17678.0,56330.0,2018-11-30,12.0,3.0,0.281434,0.281434,21010.0,39.360001
276088,17678.0,56330.0,2018-12-31,12.0,3.0,-0.320376,-0.320376,21010.0,26.750000
276089,17679.0,56331.0,2018-04-30,11.0,3.0,NaN,NaN,10272.0,26.075001


In [27]:
db.describe_table(library="crsp", table="msf")

Approximately 4509850 rows in crsp.msf.


,name,nullable,type
0,cusip,True,VARCHAR(8)
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


In [28]:
crsp_m[['permco','permno','shrcd','exchcd']]

,permco,permno,shrcd,exchcd
0,7953.0,10001.0,11.0,2.0
1,7953.0,10001.0,11.0,2.0
2,7953.0,10001.0,11.0,2.0
3,7953.0,10001.0,11.0,2.0
4,7953.0,10001.0,11.0,2.0
...,...,...,...,...
276086,56330.0,17678.0,12.0,3.0
276087,56330.0,17678.0,12.0,3.0
276088,56330.0,17678.0,12.0,3.0
276089,56331.0,17679.0,11.0,3.0


In [29]:
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)


In [30]:
crsp_m[['permco','permno','shrcd','exchcd']]

,permco,permno,shrcd,exchcd
0,7953,10001,11,2
1,7953,10001,11,2
2,7953,10001,11,2
3,7953,10001,11,2
4,7953,10001,11,2
...,...,...,...,...
276086,56330,17678,12,3
276087,56330,17678,12,3
276088,56330,17678,12,3
276089,56331,17679,11,3


In [31]:
crsp_m['date']=pd.to_datetime(crsp_m['date'])

In [32]:
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

In [33]:
crsp_m['date']

0        2015-01-30
1        2015-02-27
2        2015-03-31
3        2015-04-30
4        2015-05-29
            ...    
276086   2018-10-31
276087   2018-11-30
276088   2018-12-31
276089   2018-04-30
276090   2018-05-31
Name: date, Length: 276091, dtype: datetime64[ns]

In [34]:
crsp_m['jdate']

0        2015-01-31
1        2015-02-28
2        2015-03-31
3        2015-04-30
4        2015-05-31
            ...    
276086   2018-10-31
276087   2018-11-30
276088   2018-12-31
276089   2018-04-30
276090   2018-05-31
Name: jdate, Length: 276091, dtype: datetime64[ns]

In [35]:
MonthEnd(0)

<0 * MonthEnds>

In [36]:
dlret = db.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)

In [37]:
dlret.permno=dlret.permno.astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

In [38]:
crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])

In [39]:
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,10488.0,9.750000,2015-01-31,NaN,NaT
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,10488.0,10.100000,2015-02-28,NaN,NaT
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,10452.0,9.960000,2015-03-31,NaN,NaT
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,10452.0,10.070000,2015-04-30,NaN,NaT
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,10488.0,10.000000,2015-05-31,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,21010.0,30.715599,2018-10-31,NaN,NaT
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,21010.0,39.360001,2018-11-30,NaN,NaT
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,21010.0,26.750000,2018-12-31,NaN,2018-12-31
276089,17679,56331,2018-04-30,11,3,NaN,NaN,10272.0,26.075001,2018-04-30,NaN,NaT


In [40]:
crsp['dlret']=crsp['dlret'].fillna(0)

In [41]:
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,10488.0,9.750000,2015-01-31,0.0,NaT
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,10488.0,10.100000,2015-02-28,0.0,NaT
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,10452.0,9.960000,2015-03-31,0.0,NaT
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,10452.0,10.070000,2015-04-30,0.0,NaT
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,10488.0,10.000000,2015-05-31,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,21010.0,30.715599,2018-10-31,0.0,NaT
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,21010.0,39.360001,2018-11-30,0.0,NaT
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,21010.0,26.750000,2018-12-31,0.0,2018-12-31
276089,17679,56331,2018-04-30,11,3,NaN,NaN,10272.0,26.075001,2018-04-30,0.0,NaT


In [42]:
crsp['ret']=crsp['ret'].fillna(0)

In [43]:
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,10488.0,9.750000,2015-01-31,0.0,NaT
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,10488.0,10.100000,2015-02-28,0.0,NaT
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,10452.0,9.960000,2015-03-31,0.0,NaT
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,10452.0,10.070000,2015-04-30,0.0,NaT
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,10488.0,10.000000,2015-05-31,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,21010.0,30.715599,2018-10-31,0.0,NaT
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,21010.0,39.360001,2018-11-30,0.0,NaT
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,21010.0,26.750000,2018-12-31,0.0,2018-12-31
276089,17679,56331,2018-04-30,11,3,0.000000,NaN,10272.0,26.075001,2018-04-30,0.0,NaT


In [44]:
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

In [45]:
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt,retadj
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,10488.0,9.750000,2015-01-31,0.0,NaT,-0.115245
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,10488.0,10.100000,2015-02-28,0.0,NaT,0.035897
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,10452.0,9.960000,2015-03-31,0.0,NaT,-0.000495
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,10452.0,10.070000,2015-04-30,0.0,NaT,0.011044
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,10488.0,10.000000,2015-05-31,0.0,NaT,-0.006951
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,21010.0,30.715599,2018-10-31,0.0,NaT,-0.212421
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,21010.0,39.360001,2018-11-30,0.0,NaT,0.281434
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,21010.0,26.750000,2018-12-31,0.0,2018-12-31,-0.320376
276089,17679,56331,2018-04-30,11,3,0.000000,NaN,10272.0,26.075001,2018-04-30,0.0,NaT,0.000000


In [46]:
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 

In [47]:
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,jdate,dlret,dlstdt,retadj,me
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,10488.0,9.750000,2015-01-31,0.0,NaT,-0.115245,102258.000000
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,10488.0,10.100000,2015-02-28,0.0,NaT,0.035897,105928.804001
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,10452.0,9.960000,2015-03-31,0.0,NaT,-0.000495,104101.920399
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,10452.0,10.070000,2015-04-30,0.0,NaT,0.011044,105251.636810
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,10488.0,10.000000,2015-05-31,0.0,NaT,-0.006951,104880.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,21010.0,30.715599,2018-10-31,0.0,NaT,-0.212421,645334.736252
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,21010.0,39.360001,2018-11-30,0.0,NaT,0.281434,826953.612823
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,21010.0,26.750000,2018-12-31,0.0,2018-12-31,-0.320376,562017.500000
276089,17679,56331,2018-04-30,11,3,0.000000,NaN,10272.0,26.075001,2018-04-30,0.0,NaT,0.000000,267842.407837


In [48]:
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,102258.000000
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,105928.804001
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,104101.920399
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,105251.636810
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,104880.000000
...,...,...,...,...,...,...,...,...,...,...
276086,17678,56330,2018-10-31,12,3,-0.212421,-0.212421,2018-10-31,-0.212421,645334.736252
276087,17678,56330,2018-11-30,12,3,0.281434,0.281434,2018-11-30,0.281434,826953.612823
276088,17678,56330,2018-12-31,12,3,-0.320376,-0.320376,2018-12-31,-0.320376,562017.500000
276089,17679,56331,2018-04-30,11,3,0.000000,NaN,2018-04-30,0.000000,267842.407837


In [49]:
crsp=crsp.sort_values(by=['jdate','permco','me'])
crsp

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
77474,15580,5,2015-01-30,11,3,-0.009926,-0.009926,2015-01-31,-0.009926,8.212218e+04
53152,14593,7,2015-01-30,11,3,0.061424,0.061424,2015-01-31,0.061424,6.824275e+08
235395,90562,33,2015-01-30,11,1,-0.187176,-0.187176,2015-01-31,-0.187176,4.494096e+05
123710,59248,33,2015-01-30,11,1,0.018921,0.018921,2015-01-31,0.018921,1.365495e+07
1848,10656,37,2015-01-30,11,3,-0.105991,-0.105991,2015-01-31,-0.105991,5.646370e+05
...,...,...,...,...,...,...,...,...,...,...
100811,18314,56613,2018-12-31,12,3,0.000000,NaN,2018-12-31,0.000000,6.798213e+04
100812,18317,56614,2018-12-31,31,3,0.000000,NaN,2018-12-31,0.000000,4.807046e+04
100815,18321,56616,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,5.253974e+05
100816,18322,56617,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,8.619975e+04


In [50]:
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
crsp_summe

,jdate,permco,me
0,2015-01-31,5,8.212218e+04
1,2015-01-31,7,6.824275e+08
2,2015-01-31,33,1.410436e+07
3,2015-01-31,37,5.646370e+05
4,2015-01-31,59,2.448127e+05
...,...,...,...
262480,2018-12-31,56613,6.798213e+04
262481,2018-12-31,56614,4.807046e+04
262482,2018-12-31,56616,5.253974e+05
262483,2018-12-31,56617,8.619975e+04


In [51]:
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
crsp_maxme

,jdate,permco,me
0,2015-01-31,5,8.212218e+04
1,2015-01-31,7,6.824275e+08
2,2015-01-31,33,1.365495e+07
3,2015-01-31,37,5.646370e+05
4,2015-01-31,59,2.448127e+05
...,...,...,...
262480,2018-12-31,56613,6.798213e+04
262481,2018-12-31,56614,4.807046e+04
262482,2018-12-31,56616,5.253974e+05
262483,2018-12-31,56617,8.619975e+04


In [52]:
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
crsp1

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
0,15580,5,2015-01-30,11,3,-0.009926,-0.009926,2015-01-31,-0.009926,8.212218e+04
1,14593,7,2015-01-30,11,3,0.061424,0.061424,2015-01-31,0.061424,6.824275e+08
2,59248,33,2015-01-30,11,1,0.018921,0.018921,2015-01-31,0.018921,1.365495e+07
3,10656,37,2015-01-30,11,3,-0.105991,-0.105991,2015-01-31,-0.105991,5.646370e+05
4,61487,59,2015-01-30,11,2,0.161962,0.161962,2015-01-31,0.161962,2.448127e+05
...,...,...,...,...,...,...,...,...,...,...
262480,18314,56613,2018-12-31,12,3,0.000000,NaN,2018-12-31,0.000000,6.798213e+04
262481,18317,56614,2018-12-31,31,3,0.000000,NaN,2018-12-31,0.000000,4.807046e+04
262482,18321,56616,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,5.253974e+05
262483,18322,56617,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,8.619975e+04


In [53]:
crsp1=crsp1.drop(['me'], axis=1)
crsp1

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj
0,15580,5,2015-01-30,11,3,-0.009926,-0.009926,2015-01-31,-0.009926
1,14593,7,2015-01-30,11,3,0.061424,0.061424,2015-01-31,0.061424
2,59248,33,2015-01-30,11,1,0.018921,0.018921,2015-01-31,0.018921
3,10656,37,2015-01-30,11,3,-0.105991,-0.105991,2015-01-31,-0.105991
4,61487,59,2015-01-30,11,2,0.161962,0.161962,2015-01-31,0.161962
...,...,...,...,...,...,...,...,...,...
262480,18314,56613,2018-12-31,12,3,0.000000,NaN,2018-12-31,0.000000
262481,18317,56614,2018-12-31,31,3,0.000000,NaN,2018-12-31,0.000000
262482,18321,56616,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000
262483,18322,56617,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000


In [54]:
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
0,15580,5,2015-01-30,11,3,-0.009926,-0.009926,2015-01-31,-0.009926,8.212218e+04
1,14593,7,2015-01-30,11,3,0.061424,0.061424,2015-01-31,0.061424,6.824275e+08
2,59248,33,2015-01-30,11,1,0.018921,0.018921,2015-01-31,0.018921,1.410436e+07
3,10656,37,2015-01-30,11,3,-0.105991,-0.105991,2015-01-31,-0.105991,5.646370e+05
4,61487,59,2015-01-30,11,2,0.161962,0.161962,2015-01-31,0.161962,2.448127e+05
...,...,...,...,...,...,...,...,...,...,...
262480,18314,56613,2018-12-31,12,3,0.000000,NaN,2018-12-31,0.000000,6.798213e+04
262481,18317,56614,2018-12-31,31,3,0.000000,NaN,2018-12-31,0.000000,4.807046e+04
262482,18321,56616,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,5.253974e+05
262483,18322,56617,2018-12-31,11,3,0.000000,NaN,2018-12-31,0.000000,8.619975e+04


In [55]:
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05
...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07


In [56]:
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})
decme

,permno,date,jdate,dec_me,year
62138,10001,2015-12-31,2015-12-31,7.826225e+04,2015
127929,10001,2016-12-30,2016-12-31,1.320260e+05,2016
62139,10025,2015-12-31,2015-12-31,3.936965e+05,2015
127930,10025,2016-12-30,2016-12-31,5.937354e+05,2016
62140,10026,2015-12-31,2015-12-31,2.179046e+06,2015
...,...,...,...,...,...
260433,93434,2018-12-31,2018-12-31,6.011915e+04,2018
65717,93436,2015-12-31,2015-12-31,3.154331e+07,2015
131259,93436,2016-12-30,2016-12-31,3.452397e+07,2016
195697,93436,2017-12-29,2017-12-31,5.255495e+07,2017


In [57]:
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013


In [58]:
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755,0.884755
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897,0.916515
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139,0.903811
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044,0.913793
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049,0.907441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806,0.879603
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710,0.772037
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011,0.983584
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013,1.021957


In [59]:
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755,0.884755,NaN
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897,0.916515,0.884755
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139,0.903811,0.916515
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044,0.913793,0.903811
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049,0.907441,0.913793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806,0.879603,0.869340
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710,0.772037,0.879603
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011,0.983584,0.772037
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013,1.021957,0.983584


In [60]:
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755,0.884755,NaN,NaN
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897,0.916515,0.884755,1.022580e+05
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139,0.903811,0.916515,1.059288e+05
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044,0.913793,0.903811,1.041019e+05
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049,0.907441,0.913793,1.052516e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806,0.879603,0.869340,5.086060e+07
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710,0.772037,0.879603,5.146109e+07
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011,0.983584,0.772037,4.542871e+07
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013,1.021957,0.983584,5.792898e+07


In [61]:
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2


,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755,0.884755,NaN,NaN,0
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897,0.916515,0.884755,1.022580e+05,1
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139,0.903811,0.916515,1.059288e+05,2
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044,0.913793,0.903811,1.041019e+05,3
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049,0.907441,0.913793,1.052516e+05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806,0.879603,0.869340,5.086060e+07,43
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710,0.772037,0.879603,5.146109e+07,44
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011,0.983584,0.772037,4.542871e+07,45
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013,1.021957,0.983584,5.792898e+07,46


In [62]:
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])
crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count
495,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,2015,1,2014-07-31,2014,7,0.884755,0.884755,NaN,1.155778e+05,0
6088,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,2015,2,2014-08-31,2014,8,1.035897,0.916515,0.884755,1.022580e+05,1
11672,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,2015,3,2014-09-30,2014,9,0.986139,0.903811,0.916515,1.059288e+05,2
17247,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,2015,4,2014-10-31,2014,10,1.011044,0.913793,0.903811,1.041019e+05,3
22826,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,2015,5,2014-11-30,2014,11,0.993049,0.907441,0.913793,1.052516e+05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238778,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,2018,8,2018-02-28,2018,2,1.011806,0.879603,0.869340,5.086060e+07,43
244187,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,2018,9,2018-03-31,2018,3,0.877710,0.772037,0.879603,5.146109e+07,44
249609,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,2018,10,2018-04-30,2018,4,1.274011,0.983584,0.772037,4.542871e+07,45
255039,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,2018,11,2018-05-31,2018,5,1.039013,1.021957,0.983584,5.792898e+07,46


In [63]:
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})
mebase

,permno,ffyear,mebase
34040,10001,2015,1.080779e+05
100797,10001,2016,7.347888e+04
165645,10001,2017,1.359710e+05
34041,10025,2015,2.816856e+05
100798,10025,2016,4.114724e+05
...,...,...,...
233354,93434,2018,8.447075e+04
37709,93436,2015,3.409612e+07
104220,93436,2016,3.142062e+07
168876,93436,2017,6.033933e+07


In [64]:
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count,mebase
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,...,1,2014-07-31,2014,7,0.884755,0.884755,NaN,1.155778e+05,0,NaN
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,...,2,2014-08-31,2014,8,1.035897,0.916515,0.884755,1.022580e+05,1,NaN
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,...,3,2014-09-30,2014,9,0.986139,0.903811,0.916515,1.059288e+05,2,NaN
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,...,4,2014-10-31,2014,10,1.011044,0.913793,0.903811,1.041019e+05,3,NaN
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,...,5,2014-11-30,2014,11,0.993049,0.907441,0.913793,1.052516e+05,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262480,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,...,8,2018-02-28,2018,2,1.011806,0.879603,0.869340,5.086060e+07,43,5.847846e+07
262481,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,...,9,2018-03-31,2018,3,0.877710,0.772037,0.879603,5.146109e+07,44,5.847846e+07
262482,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,...,10,2018-04-30,2018,4,1.274011,0.983584,0.772037,4.542871e+07,45,5.847846e+07
262483,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,...,11,2018-05-31,2018,5,1.039013,1.021957,0.983584,5.792898e+07,46,5.847846e+07


In [65]:
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])
crsp3

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count,mebase,wt
0,10001,7953,2015-01-30,11,2,-0.115245,-0.115245,2015-01-31,-0.115245,1.022580e+05,...,2014-07-31,2014,7,0.884755,0.884755,NaN,1.155778e+05,0,NaN,NaN
1,10001,7953,2015-02-27,11,2,0.035897,0.035897,2015-02-28,0.035897,1.059288e+05,...,2014-08-31,2014,8,1.035897,0.916515,0.884755,1.022580e+05,1,NaN,NaN
2,10001,7953,2015-03-31,11,2,-0.000495,-0.013861,2015-03-31,-0.000495,1.041019e+05,...,2014-09-30,2014,9,0.986139,0.903811,0.916515,1.059288e+05,2,NaN,NaN
3,10001,7953,2015-04-30,11,2,0.011044,0.011044,2015-04-30,0.011044,1.052516e+05,...,2014-10-31,2014,10,1.011044,0.913793,0.903811,1.041019e+05,3,NaN,NaN
4,10001,7953,2015-05-29,11,2,-0.006951,-0.006951,2015-05-31,-0.006951,1.048800e+05,...,2014-11-30,2014,11,0.993049,0.907441,0.913793,1.052516e+05,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262480,93436,53453,2018-08-31,11,3,0.011806,0.011806,2018-08-31,0.011806,5.146109e+07,...,2018-02-28,2018,2,1.011806,0.879603,0.869340,5.086060e+07,43,5.847846e+07,5.083764e+07
262481,93436,53453,2018-09-28,11,3,-0.122290,-0.122290,2018-09-30,-0.122290,4.542871e+07,...,2018-03-31,2018,3,0.877710,0.772037,0.879603,5.146109e+07,44,5.847846e+07,5.143786e+07
262482,93436,53453,2018-10-31,11,3,0.274011,0.274011,2018-10-31,0.274011,5.792898e+07,...,2018-04-30,2018,4,1.274011,0.983584,0.772037,4.542871e+07,45,5.847846e+07,4.514752e+07
262483,93436,53453,2018-11-30,11,3,0.039013,0.039013,2018-11-30,0.039013,6.018898e+07,...,2018-05-31,2018,5,1.039013,1.021957,0.983584,5.792898e+07,46,5.847846e+07,5.751846e+07


In [66]:
decme

,permno,date,jdate,dec_me,year
62138,10001,2015-12-31,2015-12-31,7.826225e+04,2015
127929,10001,2016-12-30,2016-12-31,1.320260e+05,2016
62139,10025,2015-12-31,2015-12-31,3.936965e+05,2015
127930,10025,2016-12-30,2016-12-31,5.937354e+05,2016
62140,10026,2015-12-31,2015-12-31,2.179046e+06,2015
...,...,...,...,...,...
260433,93434,2018-12-31,2018-12-31,6.011915e+04,2018
65717,93436,2015-12-31,2015-12-31,3.154331e+07,2015
131259,93436,2016-12-30,2016-12-31,3.452397e+07,2016
195697,93436,2017-12-29,2017-12-31,5.255495e+07,2017


In [67]:
decme['year']=decme['year']+1
decme

,permno,date,jdate,dec_me,year
62138,10001,2015-12-31,2015-12-31,7.826225e+04,2016
127929,10001,2016-12-30,2016-12-31,1.320260e+05,2017
62139,10025,2015-12-31,2015-12-31,3.936965e+05,2016
127930,10025,2016-12-30,2016-12-31,5.937354e+05,2017
62140,10026,2015-12-31,2015-12-31,2.179046e+06,2016
...,...,...,...,...,...
260433,93434,2018-12-31,2018-12-31,6.011915e+04,2019
65717,93436,2015-12-31,2015-12-31,3.154331e+07,2016
131259,93436,2016-12-30,2016-12-31,3.452397e+07,2017
195697,93436,2017-12-29,2017-12-31,5.255495e+07,2018


In [68]:
decme=decme[['permno','year','dec_me']]
decme

,permno,year,dec_me
62138,10001,2016,7.826225e+04
127929,10001,2017,1.320260e+05
62139,10025,2016,3.936965e+05
127930,10025,2017,5.937354e+05
62140,10026,2016,2.179046e+06
...,...,...,...
260433,93434,2019,6.011915e+04
65717,93436,2016,3.154331e+07
131259,93436,2017,3.452397e+07
195697,93436,2018,5.255495e+07


In [69]:
crsp3_jun = crsp3[crsp3['month']==6]
crsp3_jun


,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count,mebase,wt
5,10001,7953,2015-06-30,11,2,0.030000,0.030000,2015-06-30,0.030000,1.080779e+05,...,2014-12-31,2014,12,1.030000,0.934664,0.907441,1.048800e+05,5,NaN,NaN
17,10001,7953,2016-06-30,11,2,-0.021008,-0.021008,2016-06-30,-0.021008,7.347888e+04,...,2015-12-31,2015,12,0.978992,0.678641,0.693204,7.505568e+04,17,1.080779e+05,7.492002e+04
29,10001,7953,2017-06-30,11,2,0.023622,0.017717,2017-06-30,0.023622,1.359710e+05,...,2016-12-31,2016,12,1.017717,1.849070,1.816881,1.336040e+05,29,7.347888e+04,1.335024e+05
36,10025,7975,2015-06-30,11,3,0.103338,0.103338,2015-06-30,0.103338,2.816856e+05,...,2014-12-31,2014,12,1.103338,0.949269,0.860361,2.553031e+05,5,NaN,NaN
48,10025,7975,2016-06-30,11,3,0.340777,0.340777,2016-06-30,0.340777,4.114724e+05,...,2015-12-31,2015,12,1.340777,1.457609,1.087138,3.068911e+05,17,2.816856e+05,3.062310e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262430,93434,53427,2018-06-29,11,3,-0.057971,-0.057971,2018-06-30,-0.057971,8.447075e+04,...,2017-12-31,2017,12,0.942029,0.783133,0.831325,8.398335e+04,41,7.461700e+04,6.203100e+04
262442,93436,53453,2015-06-30,11,3,0.069617,0.069617,2015-06-30,0.069617,3.409612e+07,...,2014-12-31,2014,12,1.069617,1.206151,1.127647,3.170212e+07,5,NaN,NaN
262454,93436,53453,2016-06-30,11,3,-0.049053,-0.049053,2016-06-30,-0.049053,3.142062e+07,...,2015-12-31,2015,12,0.950947,0.791322,0.832140,3.293602e+07,17,3.409612e+07,2.837276e+07
262466,93436,53453,2017-06-30,11,3,0.060409,0.060409,2017-06-30,0.060409,6.033933e+07,...,2016-12-31,2016,12,1.060409,1.703458,1.606416,5.601430e+07,29,3.142062e+07,5.047460e+07


In [111]:
list(crsp3_jun.columns) 

['permno',
 'permco',
 'date',
 'shrcd',
 'exchcd',
 'ret',
 'retx',
 'jdate',
 'retadj',
 'me',
 'year',
 'month',
 'ffdate',
 'ffyear',
 'ffmonth',
 '1+retx',
 'cumretx',
 'lcumretx',
 'lme',
 'count',
 'mebase',
 'wt']

In [70]:
crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])
crsp_jun

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count,mebase,wt,dec_me
0,10001,7953,2016-06-30,11,2,-0.021008,-0.021008,2016-06-30,-0.021008,7.347888e+04,...,2015,12,0.978992,0.678641,0.693204,7.505568e+04,17,1.080779e+05,7.492002e+04,7.826225e+04
1,10001,7953,2017-06-30,11,2,0.023622,0.017717,2017-06-30,0.023622,1.359710e+05,...,2016,12,1.017717,1.849070,1.816881,1.336040e+05,29,7.347888e+04,1.335024e+05,1.320260e+05
2,10025,7975,2016-06-30,11,3,0.340777,0.340777,2016-06-30,0.340777,4.114724e+05,...,2015,12,1.340777,1.457609,1.087138,3.068911e+05,17,2.816856e+05,3.062310e+05,3.936965e+05
3,10026,7976,2016-06-30,11,3,0.134218,0.130521,2016-06-30,0.134218,2.220569e+06,...,2015,12,1.130521,1.077708,0.953285,1.964304e+06,17,2.068644e+06,1.972006e+06,2.179046e+06
4,10026,7976,2017-06-30,11,3,0.018370,0.015142,2017-06-30,0.018370,2.473539e+06,...,2016,12,1.015142,1.107320,1.090802,2.435212e+06,29,2.220569e+06,2.422202e+06,2.494741e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15832,93434,53427,2017-06-30,11,3,0.024691,0.024691,2017-06-30,0.024691,7.461700e+04,...,2016,12,1.024691,0.951835,0.928899,7.281900e+04,29,7.438596e+04,6.909705e+04,8.256080e+04
15833,93434,53427,2018-06-29,11,3,-0.057971,-0.057971,2018-06-30,-0.057971,8.447075e+04,...,2017,12,0.942029,0.783133,0.831325,8.398335e+04,41,7.461700e+04,6.203100e+04,9.487920e+04
15834,93436,53453,2016-06-30,11,3,-0.049053,-0.049053,2016-06-30,-0.049053,3.142062e+07,...,2015,12,0.950947,0.791322,0.832140,3.293602e+07,17,3.409612e+07,2.837276e+07,3.154331e+07
15835,93436,53453,2017-06-30,11,3,0.060409,0.060409,2017-06-30,0.060409,6.033933e+07,...,2016,12,1.060409,1.703458,1.606416,5.601430e+07,29,3.142062e+07,5.047460e+07,3.452397e+07


In [81]:
ccm=db.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

In [82]:
ccm

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,None
...,...,...,...,...,...,...
32741,324684,17808.0,LC,P,2018-05-31,None
32742,326688,17676.0,LC,P,2018-05-02,None
32743,327451,17884.0,LC,P,2018-06-18,None
32744,328032,18313.0,LC,P,2018-12-10,None


In [84]:
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
ccm

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,NaT
...,...,...,...,...,...,...
32741,324684,17808.0,LC,P,2018-05-31,NaT
32742,326688,17676.0,LC,P,2018-05-02,NaT
32743,327451,17884.0,LC,P,2018-06-18,NaT
32744,328032,18313.0,LC,P,2018-12-10,NaT


In [87]:
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date


In [100]:
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))
ccm['linkenddt'].dtype

dtype('<M8[ns]')

In [102]:
ccm['linkenddt']=ccm['linkenddt'].dt.date
ccm

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,2019-09-27
...,...,...,...,...,...,...
32741,324684,17808.0,LC,P,2018-05-31,2019-09-27
32742,326688,17676.0,LC,P,2018-05-02,2019-09-27
32743,327451,17884.0,LC,P,2018-06-18,2019-09-27
32744,328032,18313.0,LC,P,2018-12-10,2019-09-27


In [103]:
ccm1=pd.merge(comp[['gvkey','datadate','be', 'count']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

In [104]:
ccm1

,gvkey,datadate,be,count,permno,linktype,linkprim,linkdt,linkenddt,yearend,jdate
0,001004,2015-05-31,949.700,0,54594.0,LU,P,1972-04-24,2019-09-27,2015-12-31,2016-06-30
1,001004,2016-05-31,900.100,1,54594.0,LU,P,1972-04-24,2019-09-27,2016-12-31,2017-06-30
2,001004,2017-05-31,951.400,2,54594.0,LU,P,1972-04-24,2019-09-27,2017-12-31,2018-06-30
3,001004,2018-05-31,952.000,3,54594.0,LU,P,1972-04-24,2019-09-27,2018-12-31,2019-06-30
4,001004,2019-05-31,905.900,4,54594.0,LU,P,1972-04-24,2019-09-27,2019-12-31,2020-06-30
...,...,...,...,...,...,...,...,...,...,...,...
49892,328795,2016-12-31,1342.700,1,18144.0,LC,P,2018-11-01,2019-09-27,2016-12-31,2017-06-30
49893,328795,2017-12-31,1418.900,2,18144.0,LC,P,2018-11-01,2019-09-27,2017-12-31,2018-06-30
49894,328795,2018-12-31,1742.800,3,18144.0,LC,P,2018-11-01,2019-09-27,2018-12-31,2019-06-30
49895,330227,2017-09-30,8.494,0,NaN,NaN,NaN,NaT,NaN,2017-12-31,2018-06-30


In [112]:
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
ccm2=ccm2[['gvkey','permno','datadate','yearend', 'jdate','be', 'count']]
ccm2

,gvkey,permno,datadate,yearend,jdate,be,count
0,001004,54594.0,2015-05-31,2015-12-31,2016-06-30,949.700,0
1,001004,54594.0,2016-05-31,2016-12-31,2017-06-30,900.100,1
2,001004,54594.0,2017-05-31,2017-12-31,2018-06-30,951.400,2
3,001004,54594.0,2018-05-31,2018-12-31,2019-06-30,952.000,3
7,001045,21020.0,2015-12-31,2015-12-31,2016-06-30,5635.000,0
...,...,...,...,...,...,...,...
49883,326688,17676.0,2018-12-31,2018-12-31,2019-06-30,2911.900,3
49886,327451,17884.0,2017-12-31,2017-12-31,2018-06-30,319.736,2
49887,327451,17884.0,2018-12-31,2018-12-31,2019-06-30,292.503,3
49889,328032,18313.0,2018-12-31,2018-12-31,2019-06-30,182.473,1


In [116]:
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

ccm_jun

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,count_x,mebase,wt,dec_me,gvkey,datadate,yearend,be,count_y,beme
0,10001,7953,2016-06-30,11,2,-0.021008,-0.021008,2016-06-30,-0.021008,7.347888e+04,...,17,1.080779e+05,7.492002e+04,7.826225e+04,012994,2015-12-31,2015-12-31,107.784,0,1.377216
1,10001,7953,2017-06-30,11,2,0.023622,0.017717,2017-06-30,0.023622,1.359710e+05,...,29,7.347888e+04,1.335024e+05,1.320260e+05,012994,2016-12-31,2016-12-31,104.237,1,0.789519
2,10025,7975,2016-06-30,11,3,0.340777,0.340777,2016-06-30,0.340777,4.114724e+05,...,17,2.816856e+05,3.062310e+05,3.936965e+05,011903,2015-10-31,2015-12-31,111.398,0,0.282954
3,10026,7976,2016-06-30,11,3,0.134218,0.130521,2016-06-30,0.134218,2.220569e+06,...,17,2.068644e+06,1.972006e+06,2.179046e+06,012825,2015-09-30,2015-12-31,643.708,0,0.295408
4,10026,7976,2017-06-30,11,3,0.018370,0.015142,2017-06-30,0.018370,2.473539e+06,...,29,2.220569e+06,2.422202e+06,2.494741e+06,012825,2016-09-30,2016-12-31,686.160,1,0.275043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15629,93434,53427,2017-06-30,11,3,0.024691,0.024691,2017-06-30,0.024691,7.461700e+04,...,29,7.438596e+04,6.909705e+04,8.256080e+04,184259,2016-06-30,2016-12-31,67.761,1,0.820741
15630,93434,53427,2018-06-29,11,3,-0.057971,-0.057971,2018-06-30,-0.057971,8.447075e+04,...,41,7.461700e+04,6.203100e+04,9.487920e+04,184259,2017-06-30,2017-12-31,61.222,2,0.645263
15631,93436,53453,2016-06-30,11,3,-0.049053,-0.049053,2016-06-30,-0.049053,3.142062e+07,...,17,3.409612e+07,2.837276e+07,3.154331e+07,184996,2015-12-31,2015-12-31,1130.989,0,0.035855
15632,93436,53453,2017-06-30,11,3,0.060409,0.060409,2017-06-30,0.060409,6.033933e+07,...,29,3.142062e+07,5.047460e+07,3.452397e+07,184996,2016-12-31,2016-12-31,4761.695,1,0.137924


In [118]:
list(ccm_jun.columns) 


['permno',
 'permco',
 'date',
 'shrcd',
 'exchcd',
 'ret',
 'retx',
 'jdate',
 'retadj',
 'me',
 'year',
 'month',
 'ffdate',
 'ffyear',
 'ffmonth',
 '1+retx',
 'cumretx',
 'lcumretx',
 'lme',
 'count_x',
 'mebase',
 'wt',
 'dec_me',
 'gvkey',
 'datadate',
 'yearend',
 'be',
 'count_y',
 'beme']

In [120]:
nyse=ccm_jun[(ccm_jun['exchcd']==1) & (ccm_jun['beme']>0) & (ccm_jun['me']>0) & (ccm_jun['count_y']>1) & ((ccm_jun['shrcd']==10) | (ccm_jun['shrcd']==11))]
nyse

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,count_x,mebase,wt,dec_me,gvkey,datadate,yearend,be,count_y,beme
22,10104,8045,2018-06-29,11,1,-0.056935,-0.056935,2018-06-30,-0.056935,1.754097e+08,...,41,2.074132e+08,1.932658e+08,1.957204e+08,012142,2017-05-31,2017-12-31,54320.000,2,0.277539
32,10145,22168,2018-06-29,11,1,-0.026097,-0.026097,2018-06-30,-0.026097,1.075953e+08,...,41,1.016120e+08,1.127574e+08,1.160644e+08,001300,2017-12-31,2017-12-31,20170.000,2,0.173783
36,10158,53454,2018-06-29,11,1,0.004184,0.004184,2018-06-30,0.004184,3.547920e+05,...,41,2.111725e+05,3.277288e+05,2.365344e+05,185128,2017-12-31,2017-12-31,337.204,2,1.425602
54,10220,53456,2018-06-29,11,1,-0.066227,-0.066227,2018-06-30,-0.066227,6.212058e+06,...,41,4.828541e+06,6.610397e+06,6.012948e+06,185229,2017-12-31,2017-12-31,285.394,2,0.047463
98,10375,8292,2018-06-29,11,1,-0.064234,-0.064234,2018-06-30,-0.064234,4.142389e+06,...,41,2.729422e+06,4.505088e+06,3.524606e+06,015363,2017-12-31,2017-12-31,2396.936,2,0.680058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15570,93374,53407,2018-06-29,11,1,0.000384,-0.006912,2018-06-30,0.000384,5.769108e+06,...,41,4.939586e+06,5.756403e+06,6.210185e+06,184899,2017-12-31,2017-12-31,3699.262,2,0.595677
15577,93384,53413,2018-06-29,11,1,0.154696,0.154696,2018-06-30,0.154696,8.047963e+04,...,41,2.787391e+05,6.939721e+04,2.956091e+05,180562,2017-12-31,2017-12-31,125.129,2,0.423292
15609,93420,53438,2018-06-29,11,1,-0.004605,-0.004605,2018-06-30,-0.004605,4.115485e+06,...,41,1.911336e+06,3.093752e+06,2.264922e+06,184442,2017-12-31,2017-12-31,3681.612,2,1.625492
15612,93422,53439,2018-06-29,11,1,0.014061,0.014061,2018-06-30,0.014061,2.914300e+06,...,41,2.429090e+06,2.907693e+06,2.305748e+06,154357,2017-12-31,2017-12-31,4315.900,2,1.871800


In [121]:
nyse_sz=nyse.groupby(['jdate'])['me'].median().to_frame().reset_index().rename(columns={'me':'sizemedn'})
nyse_sz

,jdate,sizemedn
0,2017-06-30,3.188152e+06
1,2018-06-30,3.279043e+06


In [122]:
nyse_bm=nyse.groupby(['jdate'])['beme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_bm=nyse_bm[['jdate','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})
nyse_bm

,jdate,bm30,bm70
0,2017-06-30,0.368023,0.518372
1,2018-06-30,0.284261,0.656809


In [123]:
nyse_breaks = pd.merge(nyse_sz, nyse_bm, how='inner', on=['jdate'])
nyse_breaks

,jdate,sizemedn,bm30,bm70
0,2017-06-30,3.188152e+06,0.368023,0.518372
1,2018-06-30,3.279043e+06,0.284261,0.656809


In [124]:
ccm1_jun = pd.merge(ccm_jun, nyse_breaks, how='left', on=['jdate'])
ccm1_jun

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,...,dec_me,gvkey,datadate,yearend,be,count_y,beme,sizemedn,bm30,bm70
0,10001,7953,2016-06-30,11,2,-0.021008,-0.021008,2016-06-30,-0.021008,7.347888e+04,...,7.826225e+04,012994,2015-12-31,2015-12-31,107.784,0,1.377216,NaN,NaN,NaN
1,10001,7953,2017-06-30,11,2,0.023622,0.017717,2017-06-30,0.023622,1.359710e+05,...,1.320260e+05,012994,2016-12-31,2016-12-31,104.237,1,0.789519,3.188152e+06,0.368023,0.518372
2,10025,7975,2016-06-30,11,3,0.340777,0.340777,2016-06-30,0.340777,4.114724e+05,...,3.936965e+05,011903,2015-10-31,2015-12-31,111.398,0,0.282954,NaN,NaN,NaN
3,10026,7976,2016-06-30,11,3,0.134218,0.130521,2016-06-30,0.134218,2.220569e+06,...,2.179046e+06,012825,2015-09-30,2015-12-31,643.708,0,0.295408,NaN,NaN,NaN
4,10026,7976,2017-06-30,11,3,0.018370,0.015142,2017-06-30,0.018370,2.473539e+06,...,2.494741e+06,012825,2016-09-30,2016-12-31,686.160,1,0.275043,3.188152e+06,0.368023,0.518372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15629,93434,53427,2017-06-30,11,3,0.024691,0.024691,2017-06-30,0.024691,7.461700e+04,...,8.256080e+04,184259,2016-06-30,2016-12-31,67.761,1,0.820741,3.188152e+06,0.368023,0.518372
15630,93434,53427,2018-06-29,11,3,-0.057971,-0.057971,2018-06-30,-0.057971,8.447075e+04,...,9.487920e+04,184259,2017-06-30,2017-12-31,61.222,2,0.645263,3.279043e+06,0.284261,0.656809
15631,93436,53453,2016-06-30,11,3,-0.049053,-0.049053,2016-06-30,-0.049053,3.142062e+07,...,3.154331e+07,184996,2015-12-31,2015-12-31,1130.989,0,0.035855,NaN,NaN,NaN
15632,93436,53453,2017-06-30,11,3,0.060409,0.060409,2017-06-30,0.060409,6.033933e+07,...,3.452397e+07,184996,2016-12-31,2016-12-31,4761.695,1,0.137924,3.188152e+06,0.368023,0.518372
